In [2]:
import os, io
import pandas as pd
import pysam
import pickle
import pybedtools
pybedtools.helpers.set_tempdir('/data/projects/temp')
from IPython.display import display, HTML

In [7]:
df_donor = pd.read_csv("/data/private/pdutta/DNABERT_data/Splice_sites/Donor_raw_files/Donor_RAW_Combined.tsv", sep="\t")
df_donor['start'] = df_donor['coordinates']-39
df_donor['end'] = df_donor['coordinates']+40

In [8]:
df_donor

,Unnamed: 0,transcript_id,exon_id,transcript_type,strand,coordinates,sequence,chr_name,Donor_80BP,start,end
0,0,ENST00000003583.12,ENSE00001879255.1,protein_coding,-,24413673,AAATTCGGCCGGCTTCTCTGCGGACTTTGAAACGCCTCTAATGATT...,chr1,AGGTTGGGCTGGCCAGATTCAGCTGCCGGGACCGGACCAGGTAGGA...,24413634,24413713
1,1,ENST00000003583.12,ENSE00003692979.1,protein_coding,-,24401318,CCATTTGCAATAATTAACCAATGTGAAATGATCTCTACAAATGCTG...,chr1,ACTGGCAAACATCCCAGACGTGCCAGTGAAGTACAGAAAGGTACAT...,24401279,24401358
2,2,ENST00000003583.12,ENSE00003638911.1,protein_coding,-,24383901,GGATTACAGTAAGGCTTAGTGTTAATGGATGTAAAGCCTGTGGAAA...,chr1,CTCATTGTCCAAGAAAGGAACTTGCATGTTTCCCTCAATGGTGAGA...,24383862,24383941
3,3,ENST00000003583.12,ENSE00003688858.1,protein_coding,-,24379652,TGCTGATTCCTTTGGTTGTCTCCTTACAGTGCGCCCGATTGGACAC...,chr1,AGCTCTCAAGTTTGAAACTCCTGCACCAAACTATTACAATGTAAGA...,24379613,24379692
4,4,ENST00000003583.12,ENSE00003631141.1,protein_coding,-,24373701,TTTTACGTAGATTTGCCACTTGTCCAGGCATGATGACGAAGACAAA...,chr1,AGAGGATCTTTCGCTTTTGCTGATAAAGGACCTCCCCCAGGTAAGC...,24373662,24373741
...,...,...,...,...,...,...,...,...,...,...,...
1242776,1286988,ENST00000684226.1,ENSE00003771339.1,non_stop_decay,-,13298956,TTTCTTTTTCAGAAATAATTTGAATTTTTTAATGAGTTCTTGGTGG...,chrY,TGGTGCAATAACATTGCCTGGAATGTTGGTCCACTTACAGGTATTC...,13298917,13298996
1242777,1286989,ENST00000684226.1,ENSE00003767626.1,non_stop_decay,-,13297706,ATCTTCATGTGTTCTATTACTTTGGGTTACTTTATGACTGTTGCTT...,chrY,ATATCAAAGTCTCAGATCCAAAGCTTTTTGAAATGATTAAGTAAGT...,13297667,13297746
1242778,1286990,ENST00000684226.1,ENSE00003761778.1,non_stop_decay,-,13260277,TATTTCTAATTGGTATACCTTTTATGTAAAATATTCAGTAGTTACT...,chrY,GACAAATGATGAACCAGCTCATTACTGTAGCATTTGTGAGGTGAGT...,13260238,13260317
1242779,1286991,ENST00000684226.1,ENSE00001713248.1,non_stop_decay,-,13251016,CTTTTTGTTTATTTATTTATTTAATATAGGTGGAGGTTTTTAATCT...,chrY,AATGGAGGACCTAATCCAAGTTTATGATCAATTTACACTAGTAAGT...,13250977,13251056


In [16]:
df_donor['Donor_80BP'].iloc[0]

'AGGTTGGGCTGGCCAGATTCAGCTGCCGGGACCGGACCAGGTAGGAAGCAGTTCCGGCGGGGGGACCGGTCCCTGAGGTC'

In [17]:
df_donor_bed = df_donor[['chr_name', 'start',	'end', 'strand','transcript_id','exon_id','transcript_type']]

In [18]:
df_donor_bed

,chr_name,start,end,strand,transcript_id,exon_id,transcript_type
0,chr1,24413634,24413713,-,ENST00000003583.12,ENSE00001879255.1,protein_coding
1,chr1,24401279,24401358,-,ENST00000003583.12,ENSE00003692979.1,protein_coding
2,chr1,24383862,24383941,-,ENST00000003583.12,ENSE00003638911.1,protein_coding
3,chr1,24379613,24379692,-,ENST00000003583.12,ENSE00003688858.1,protein_coding
4,chr1,24373662,24373741,-,ENST00000003583.12,ENSE00003631141.1,protein_coding
...,...,...,...,...,...,...,...
1242776,chrY,13298917,13298996,-,ENST00000684226.1,ENSE00003771339.1,non_stop_decay
1242777,chrY,13297667,13297746,-,ENST00000684226.1,ENSE00003767626.1,non_stop_decay
1242778,chrY,13260238,13260317,-,ENST00000684226.1,ENSE00003761778.1,non_stop_decay
1242779,chrY,13250977,13251056,-,ENST00000684226.1,ENSE00001713248.1,non_stop_decay


In [19]:
df_donor_unique_bed = df_donor_bed.groupby(['chr_name','start', 'end']).agg(lambda x: ", ".join(x.unique())).reset_index()
df_donor_unique_bed

,chr_name,start,end,strand,transcript_id,exon_id,transcript_type
0,chr1,12189,12268,+,"ENST00000450305.2, ENST00000456328.2","ENSE00001671638.2, ENSE00002234944.1","transcribed_unprocessed_pseudogene, processed_..."
1,chr1,12659,12738,+,ENST00000450305.2,ENSE00001758273.2,transcribed_unprocessed_pseudogene
2,chr1,12683,12762,+,ENST00000456328.2,ENSE00003582793.1,processed_transcript
3,chr1,13336,13415,+,ENST00000450305.2,ENSE00001746346.2,transcribed_unprocessed_pseudogene
4,chr1,15756,15835,-,ENST00000488147.1,ENSE00002030414.1,unprocessed_pseudogene
...,...,...,...,...,...,...,...
297951,chrY,57211531,57211610,+,ENST00000464205.6_PAR_Y,ENSE00003785071.1,processed_transcript
297952,chrY,57211582,57211661,+,"ENST00000340131.12_PAR_Y, ENST00000359512.8_PA...","ENSE00003627301.1, ENSE00003905275.1, ENSE0000...","retained_intron, protein_coding"
297953,chrY,57213164,57213243,-,ENST00000507418.6_PAR_Y,ENSE00002036959.1,unprocessed_pseudogene
297954,chrY,57213840,57213919,-,ENST00000507418.6_PAR_Y,ENSE00002046926.1,unprocessed_pseudogene


In [21]:
df_donor_unique_bed.to_csv("/data/private/pdutta/DNABERT_data/Splice_sites/Donor_raw_files/Unique_Donor_RAW_Combined_BED.tsv", sep="\t", index=False)

In [22]:
with open("/data/projects/VCF_files/BRCA/Postprocessed_files/vcf_data.pkl", "rb") as file:
    loaded_dictionary = pickle.load(file)

In [23]:
donor_bed = pybedtools.BedTool.from_dataframe(df_donor_unique_bed)
vcf_donor_data = {}
index = 0
for key, value in loaded_dictionary.items():
    #display(value)
    vcf_bed = pybedtools.BedTool.from_dataframe(value)
    if ((index%200) ==0):
        print(index, key)
    index= index+1
    # Use intersect function
    intersect_vcf_donor = donor_bed.intersect(vcf_bed, wa=True, wb=True)
    column_list  = df_donor_bed.columns.to_list() + value.columns.to_list()
    #print(column_list)
    df_intersection = intersect_vcf_donor.to_dataframe(names=column_list)
    #display(df_intersection)
    vcf_donor_data[key] = df_intersection

0 c5bbc8ae-3efe-43a7-9613-ce7f967b34a7.wxs.mutect2.raw_somatic_mutation.vcf.gz
200 TCGA_BRCA.8e9c3ea3-2c4d-4d2e-add4-e8992574b45a.wxs.Pindel.somatic_annotation.vcf.gz
400 3593447c-3a0d-485b-9743-3146ccc5081a.wxs.varscan2.raw_somatic_mutation.vcf.gz
600 94068b27-d41c-4db0-8f73-3e0db65aeb20.wxs.varscan2.raw_somatic_mutation.vcf.gz
800 52b3dee4-21ae-4131-8a1d-b1a5ebc1e3c7.wxs.varscan2.raw_somatic_mutation.vcf.gz
1000 TCGA_BRCA.7e5e9662-c941-4aeb-843e-0db3946f7d98.wxs.VarScan2.somatic_annotation.vcf.gz
1200 f7116df5-11e6-4fcb-83a0-bff3f201a836.wxs.mutect2.raw_somatic_mutation.vcf.gz
1400 TCGA_BRCA.d83d5baf-04ce-459a-8131-cae34461baa2.wxs.VarScan2.somatic_annotation.vcf.gz
1600 7fa0da5a-951b-46e6-9518-b14d4a6ee85b.wxs.pindel.raw_somatic_mutation.vcf.gz
1800 4eecd5a8-0139-49e3-aa2b-5b6ffd1eed03.wxs.varscan2.raw_somatic_mutation.vcf.gz
2000 TCGA_BRCA.45acd765-57b5-4065-9593-79261328ce30.wxs.VarScan2.somatic_annotation.vcf.gz
2200 TCGA_BRCA.f97bf6d4-5d3e-480b-aeaa-92e81e41161b.wxs.MuTect2.soma

In [24]:
with open("/data/projects/VCF_files/BRCA/Postprocessed_files/intersect_donor_vcf_data.pkl", "wb") as file:
    pickle.dump(vcf_donor_data, file)